In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
from config import (census_key, gkey)
c = Census(census_key, year=2013)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [2]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"
                          ), {'for': 'state:*'})

# Convert to DataFrame

census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "State", "state": "StateNo",
                                      "zip code tabulation area": "Zipcode"})
census_pd.head()
#census_pd.dtypes()

,State,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,StateNo
0,Alabama,43253.0,4799277.0,38.1,23680.0,870631.0,241930.0,01
1,Alaska,70760.0,720316.0,33.6,32651.0,69514.0,32939.0,02
2,Arizona,49774.0,6479703.0,36.3,25358.0,1131901.0,316360.0,04
3,Arkansas,40768.0,2933369.0,37.5,22170.0,547328.0,121235.0,05
4,California,61094.0,37659181.0,35.4,29527.0,5885417.0,2168665.0,06


In [ ]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

ation"].astype(int)# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Popul

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate","Zipcode"]]

census_pd.head()

In [ ]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [ ]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("../Resources/state_centroids.csv")
centroids.head()

In [ ]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("../Resources/state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

In [ ]:
# Convert poverty rate as a list
# Convert bank rate to list
poverty_rate = census_data["Poverty Rate"].tolist()

In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

![10-State_Markers.png](../Images/10-State_Markers.png)